In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import langchain
from langchain.cache import InMemoryCache
import os
from dotenv import load_dotenv
langchain.cache = InMemoryCache()
load_dotenv()


True

In [3]:
chat = ChatOpenAI()

In [5]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    SystemMessagePromptTemplate,
)